In [0]:
# https://www.bbc.com/sport/formula1/2020/abu-dhabi-grand-prix/results

In [0]:
%run "../includes/configuration"

In [0]:
drivers_df = spark.read.parquet(f"{processed_folder_path}/drivers")\
    .withColumnRenamed("number","driver_number")\
    .withColumnRenamed("name","driver_name")\
    .withColumnRenamed("nationality","driver_nationality")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1864802303235227>:1
----> 1 drivers_df = spark.read.parquet(f"{processed_folder_path}/drivers")\
      2     .withColumnRenamed("number","driver_number")\
      3     .withColumnRenamed("name","driver_name")\
      4     .withColumnRenamed("nationality","driver_nationality")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:533, in DataFrameReader.parquet(self, *paths, **options)
    522 int96RebaseMode = options.get("int96RebaseMode", None)
    523 

In [0]:
constructors_df = spark.read.parquet(f"{processed_folder_path}/constructors")\
    .withColumnRenamed("name","team")

In [0]:
circuits_df = spark.read.parquet(f"{processed_folder_path}/circuits")\
    .withColumnRenamed("location","circuit_location")

In [0]:
races_df = spark.read.parquet(f"{processed_folder_path}/races")\
    .withColumnRenamed("name","race_name")\
    .withColumnRenamed("race_timestamp","race_date")

In [0]:
results_df = spark.read.parquet(f"{processed_folder_path}/results")\
    .withColumnRenamed("time","race_time")

####Join Circuits to Races

In [0]:
race_circuits_df = races_df.join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner")\
    .select(races_df.race_id, races_df.race_year, races_df.race_name, races_df.race_date, circuits_df.circuit_location)

####Join Results to all other dataframes

In [0]:
race_results_df = results_df.join(race_circuits_df, results_df.race_id == race_circuits_df.race_id, "inner")\
    .join(drivers_df, results_df.driver_id == drivers_df.driver_id, "inner")\
    .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id, "inner")

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
final_df = race_results_df.select("race_year", "race_name", "race_date", "circuit_location", "driver_name", "driver_number", "driver_nationality",
                                  "team", "grid", "fastest_lap", "race_time", "points","position")\
                          .withColumn("created_date",current_timestamp())

In [0]:
display(final_df.filter("race_year = 2020 and race_name = 'Abu Dhabi Grand Prix'").orderBy(final_df.points.desc()))

In [0]:
final_df.write.mode('overwrite').parquet(f"{presentation_folder_path}/race_results")